In [1]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import mysql.connector as cnt
import random

In [ ]:
# x = eval(price['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
# dict(x)
# x['meta']['sell_price']

In [2]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 0'''.format(table=table)
    s = c.execute(sql)
    # r = s.description
    column_names = [description[0] for description in c.description]
    return column_names

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(1)
            db.commit()
            time.sleep(3)
    c.close()
    db.close()
    return True


def createMainTable(dbname:str) -> bool:
    sql = '''
    drop table if exists PlanseteVarle
    '''
    db = sqlite3.connect(dbname)
    c = db.cursor()
    c.execute(sql)
    sql = '''create table if not exists PlanseteVarle
    (
        nuoroda text,
        firstInfo text
    )
    '''
    c.execute(sql)
    c.close()
    db.close()
    return True
    pass

def writeMany(lst, table, dbname) -> bool:
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''insert into {table} values ({n})'''
    q = ['?' for i in range(0,len(lst))]
    q_ = ', '.join(q)
    r_ = sql.format(table=table, n=q_)
    # print('::PROVIDED::', lst, len(lst), '::SQL COMMAND::', r_, len(q), sep='\n')
    c.execute(r_, lst)
    db.commit()
    c.close()
    db.close()
    return True


In [9]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.varle.lt/plansetiniai-kompiuteriai/?p=1'
driver.get(url)
time.sleep(5)
source = driver.page_source


driver.close()
print('Baigta!!!')

Baigta!!!


In [10]:
bs = BeautifulSoup(source, 'html.parser')
p_nuorodos = []

linkai = bs.find_all('div', {'class':'GRID_ITEM'})
for linkas in linkai:
    # linkas = bs.find('div', {'class':'product-title'}).find('div', {'class':'product-title'}).find('a')
    link = linkas.find('div', {'class':'product-title'}).find('a')
    # print(link['href'])
    add_link = 'https://www.varle.lt'+ link['href']
    # print(add_link)
    p_nuorodos.append(add_link)
print(len(p_nuorodos))
print(p_nuorodos)

pages = []
pages_numbers = bs.find('div', {'class':'PAGINATION'}).find_all('li', {'class':'number'})
for page in pages_numbers:
    # print(page.text.strip())
    pages.append(int(page.text.strip()))
pages_max_number = max(pages)
print(pages_max_number)

38
['https://www.varle.lt/plansetiniai-kompiuteriai/plansete-lenovo-tab-m11-11-8gb-128gb-lte-wifi--36331698.html', 'https://www.varle.lt/plansetiniai-kompiuteriai/plansetinis-kompiuteris-lenovo-tab-m11-11-4gb--31830355.html', 'https://www.varle.lt/plansetiniai-kompiuteriai/plansetinis-kompiuteris-lenovo-tab-m9-9-hd-464gb--25338254.html', 'https://www.varle.lt/plansetiniai-kompiuteriai/plansetinis-kompiuteris-samsung-galaxy-tab-s9-fe-wifi--29767300.html', 'https://www.varle.lt/plansetiniai-kompiuteriai/plansetinis-kompiuteris-kruger-matz-eagle-1075--27910373.html', 'https://www.varle.lt/plansetiniai-kompiuteriai/plansetinis-kompiuteris-kruger-matz-eagle-km1076--32044689.html', 'https://www.varle.lt/plansetiniai-kompiuteriai/plansetinis-kompiuteris-kruger-matz-eagle-807-84--29070609.html', 'https://www.varle.lt/plansetiniai-kompiuteriai/plansete-samsung-galaxy-tab-a9-wifi-64-gb-11-col--31148109.html', 'https://www.varle.lt/plansetiniai-kompiuteriai/plansetinis-kompiuteris-xiaomi-redmi-pa

In [11]:
for nuoroda in p_nuorodos[:1]:
    print(nuoroda)
    opcijos = Options()
    opcijos.add_argument('--incognito')
    opcijos.add_argument('--ignore-certificate-errors')
    opcijos.add_argument('--ignore-ssl-errors')
    driver = webdriver.Chrome(options=opcijos)
    # url = nuoroda
    url = 'https://www.varle.lt/plansetiniai-kompiuteriai/plansete-samsung-galaxy-tab-a7-lite-wifi-32-gb-sm-t220n--39025925.html'
    driver.get(url)
    time.sleep(5)
    source = driver.page_source


    driver.close()

https://www.varle.lt/plansetiniai-kompiuteriai/plansete-lenovo-tab-m11-11-8gb-128gb-lte-wifi--36331698.html


In [16]:
bs = BeautifulSoup(source, 'html.parser')
pavadinimas = bs.find('div', {'class':'main'}).find('h1', {'class':'title'}).text.strip()
print(pavadinimas)

# varle_kodas = bs.find('div', {'class':'main'}).find('span', {'itemprop':'sku'}).text
# print(varle_kodas)

kaina = bs.find('span', {'class':'price-value'}).text.strip()
print(kaina)

buvusi_kaina = bs.find('span', {'class':'previous-price'})
if buvusi_kaina != None:
    buvusi_kaina = buvusi_kaina.text.strip()
else:
    buvusi_kaina = None
print(buvusi_kaina)

specifiacija_keys = []
specifiacija_values = []
try:
    specifiacija = bs.find('div', {'class':'PRODUCT_SPEC'}).find_all('li', {'class':'spec-line'})
    for line in specifiacija:
        try:
            spec_key = line.find('span', {'class':'left-column'}).text
            # print(spec_key)
            specifiacija_keys.append(spec_key)
            spec_value = line.find('span', {'class':'right-column'}).text
            # print(spec_value)
            specifiacija_values.append(spec_value)
        except:
            pass
        try:
            spec_key = line.find('div', {'class':'left-column'}).text
            # print(spec_key)
            specifiacija_keys.append(spec_key)
            spec_value = line.find('div', {'class':'right-column'}).text
            # print(spec_value)
            specifiacija_values.append(spec_value)
        except:
            pass
except:
    pass

try:
    specifiacija_k = bs.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'key'})
    for key in specifiacija_k:
        # print(key.text.strip())
        specifiacija_keys.append(key.text.strip().lower())
    specifiacija_v = bs.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'value'})
    for val in specifiacija_v:
        # print(val.text.strip())
        specifiacija_values.append(val.text.strip())       
except:
    pass

print(len(specifiacija_keys), specifiacija_keys)
print(len(specifiacija_values), specifiacija_values)

spec_filtrai_keys = []
spec_filtrai_values = []
try:
    spec_filtai = bs.find('div', {'class':'PRODUCT_SPEC_FILTERS'}).find_all('li', {'class':'spec-line'})
    for line in spec_filtai:
        spec_key = line.find('span', {'class':'left-column'}).text.strip().lower()
        # print(spec_key)
        spec_filtrai_keys.append(spec_key)
        spec_value = line.find('span', {'class':'right-column'}).text.strip()
        # print(spec_value)
        spec_filtrai_values.append(spec_value)
except:
    pass
    
print(len(spec_filtrai_keys), spec_filtrai_keys)
print(len(spec_filtrai_values), spec_filtrai_values)

# for k,v in zip(specifiacija_keys, specifiacija_values):
#     print(k, v)

Planšetė Samsung Galaxy Tab A7 Lite WiFi 32 GB SM-T220N Tamsiai pilka (Ekspozicinė prekė)
129
149 €
28 ['gross weight', 'net weight', 'producer', 'wi-fi belaidis ryšys', 'belaidžio ryšio standartai', '„bluetooth“', '„bluetooth“ versija', 'nfc', 'priekinė kamera', 'galinės kameros raiška (skaičiais)', 'galinė kamera', 'priekinės kameros raiška (skaičiais)', 'vidinė atmintis', 'suderinamos atminties kortelės', 'maksimalus atminties kortelės dydis', 'vidinės laikmenos talpa', 'gaminio spalva', 'akumuliatoriaus talpa', 'baterijos technologija', 'įdiegta operacinė sistema', 'operacinės sistemos versija', 'ekrano įstrižainė', 'ekranas', 'ekrano technologija', 'gamintojo prekės modelio pavadinimas', 'gamintojo prekės serija', 'procesoriaus modelis', 'procesoriaus dažnis']
28 ['0.648 kg', '0.448 kg', 'Samsung', 'Yes', '802.11 a/b/g/n/ac', 'Yes', '5.0', 'No', 'Yes', '8 mp', 'Yes', '2 mp', '3 gb', 'Microsdxc', '1000 gb', '32 gb', 'Grey', '5100 mah', 'Li-po', 'Android', '11.0', '8.7 col.', '1340 

In [ ]:
# Plansetiniai kompiuteriai lentele
# createMainTable('VarlePigu.db')

True

In [4]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.varle.lt/plansetiniai-kompiuteriai/?p=14'
driver.get(url)
time.sleep(5)
source = driver.page_source

bs = BeautifulSoup(source, 'html.parser')
pages = []
pages_numbers = bs.find('div', {'class':'PAGINATION'}).find_all('li', {'class':'number'})
for page in pages_numbers:
    # print(page.text.strip())
    pages.append(int(page.text.strip()))
pages_max_number = max(pages)

counts = 495
psl = 14
# iki_kokio_psl = 14
# while psl < iki_kokio_psl:
while psl < pages_max_number + 1:
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    p_nuorodos = []

    linkai = bs.find_all('div', {'class':'GRID_ITEM'})
    for linkas in linkai:
        # linkas = bs.find('div', {'class':'product-title'}).find('div', {'class':'product-title'}).find('a')
        link = linkas.find('div', {'class':'product-title'}).find('a')
        # print(link['href'])
        add_link = 'https://www.varle.lt'+ link['href']
        # print(add_link)
        p_nuorodos.append(add_link)
        
    for nuoroda in p_nuorodos:
        print(psl, nuoroda)
        url = nuoroda
        driver.get(url)
        time.sleep(random.randint(5,10))
        source = driver.page_source
        
        cols_i_have = getColNamesList(table='PlanseteVarle', dbname='VarlePigu.db')
        
        duomenys = {}
        for col in cols_i_have:
            duomenys[col] = None
            
        duomenys['nuoroda'] = nuoroda
        
        bs_preke = BeautifulSoup(source, 'html.parser')
        
        firstInfo = bs_preke.find('div', {'class':'main'}).find('h1', {'class':'title'}).text.strip()
        duomenys['firstInfo'] = firstInfo
        # print(pavadinimas)


        kaina = bs_preke.find('span', {'class':'price-value'}).text.strip()
        duomenys['kaina'] = kaina
        # print(kaina)

        buvusi_kaina = bs_preke.find('span', {'class':'previous-price'})
        if buvusi_kaina != None:
            buvusi_kaina = buvusi_kaina.text.strip()
        else:
            buvusi_kaina = None
        duomenys['buvusiKaina'] = buvusi_kaina
        # print(buvusi_kaina)

        specifiacija_keys = []
        specifiacija_values = []
        try:
            specifiacija = bs_preke.find('div', {'class':'PRODUCT_SPEC'}).find_all('li', {'class':'spec-line'})
            for line in specifiacija:
                try:
                    spec_key = line.find('span', {'class':'left-column'}).text.lower()
                    # print(spec_key)
                    specifiacija_keys.append(spec_key)
                    spec_value = line.find('span', {'class':'right-column'}).text
                    # print(spec_value)
                    specifiacija_values.append(spec_value)
                except:
                    pass
                try:
                    spec_key = line.find('div', {'class':'left-column'}).text.lower()
                    # print(spec_key)
                    specifiacija_keys.append(spec_key)
                    spec_value = line.find('div', {'class':'right-column'}).text
                    # print(spec_value)
                    specifiacija_values.append(spec_value)
                except:
                    pass
        except:
            pass

        try:
            specifiacija_k = bs_preke.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'key'})
            for key in specifiacija_k:
                # print(key.text.strip())
                specifiacija_keys.append(key.text.strip().lower())
            specifiacija_v = bs_preke.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'value'})
            for val in specifiacija_v:
                # print(val.text.strip())
                specifiacija_values.append(val.text.strip())       
        except:
            pass
        
        
        # print(len(specifiacija_keys), specifiacija_keys)
        # print(len(specifiacija_values), specifiacija_values)

        spec_filtrai_keys = []
        spec_filtrai_values = []
        try:
            spec_filtai = bs_preke.find('div', {'class':'PRODUCT_SPEC_FILTERS'}).find_all('li', {'class':'spec-line'})
            for line in spec_filtai:
                spec_key = line.find('span', {'class':'left-column'}).text.strip().lower()
                # print(spec_key)
                spec_filtrai_keys.append(spec_key)
                spec_value = line.find('span', {'class':'right-column'}).text.strip()
                # print(spec_value)
                spec_filtrai_values.append(spec_value)
                
            # print(len(spec_filtrai_keys), spec_filtrai_keys)
            # print(len(spec_filtrai_values), spec_filtrai_values)
        except:
            pass
        
        for k, v in zip(spec_filtrai_keys, spec_filtrai_values):
            duomenys[k] = v
            
        for k, v in zip(specifiacija_keys, specifiacija_values):
            duomenys[k] = v
            
        cols_i_need = list(duomenys.keys())
        
        data = tuple(duomenys.values())
        
        cols_i_have = getColNamesList(table='PlanseteVarle', dbname='VarlePigu.db')
        
        alterTableAddColumn(cols_i_need, cols_i_have, 'PlanseteVarle', 'VarlePigu.db')
        
        writeMany(data, 'PlanseteVarle', 'VarlePigu.db')
        print(f'Įrašytas įrašas Nr {counts}, puslapis {psl}')
        counts = counts +1
    
    print(f'Įrašytas puslapis {psl}')
    psl = psl +1
    # if psl < iki_kokio_psl:
    if psl <= pages_max_number:
        next_nuoroda = f'https://www.varle.lt/plansetiniai-kompiuteriai/?p={psl}'
        driver.get(next_nuoroda)
        time.sleep(3)
        print(f'kitas puslapis: {psl}')
    
driver.close()
print('Baigta!!!')


14 https://www.varle.lt/plansetiniai-kompiuteriai/plansete-lenovo-tab-m11-lte-4gb-128gb-luna-pilkas--41012016.html
Įrašytas įrašas Nr 495, puslapis 14
14 https://www.varle.lt/plansetiniai-kompiuteriai/plansete-apple-ipad-air-13-m2-wi-fi-cellular-512gb--35475567.html
Įrašytas įrašas Nr 496, puslapis 14
14 https://www.varle.lt/plansetiniai-kompiuteriai/plansete-zebra-et56-84-display-win10-intel-e3940--25073348.html
Įrašytas įrašas Nr 497, puslapis 14
14 https://www.varle.lt/plansetiniai-kompiuteriai/plansete-apple-ipad-air-13-m2-wi-fi-cellular-1tb--35475598.html
Įrašytas įrašas Nr 498, puslapis 14
14 https://www.varle.lt/plansetiniai-kompiuteriai/plansete-apple-ipad-air-11-m2-wi-fi-512gb-starlight--36899248.html
Įrašytas įrašas Nr 499, puslapis 14
14 https://www.varle.lt/plansetiniai-kompiuteriai/plansete-tab-mega1-11-256gb-ltetab-mega1-purple--41023811.html
Įrašytas įrašas Nr 500, puslapis 14
14 https://www.varle.lt/plansetiniai-kompiuteriai/plansete-tab-mega1-11-256gb-ltetab-mega1-pilk